# MCP Integration Testing

This notebook tests the Model Context Protocol (MCP) integration.

## 🔌 What is MCP?
Model Context Protocol is a standardized way to expose tools to LLMs. Our implementation:
- Wraps all tools in a unified `execute_tool` interface
- Provides consistent input/output schemas
- Enables dynamic tool registration
- Supports both direct tools and MCP servers

## 📚 What We'll Test
1. MCP Tool Registry
2. Individual MCP Tools
3. execute_tool Wrapper
4. Tool Modes (mcp vs direct)
5. MCP Protocol Compliance

**Last Updated**: January 2025

## Setup

In [ ]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)

print(f"✅ Project root: {project_root}")
print(f"✅ Current directory: {os.getcwd()}")

from config import settings
print(f"\n✅ Configuration loaded")
print(f"   Tool Mode: {settings.TOOL_MODE}")

## Test 1: MCP Tool Registry

Test the central registry for all MCP tools

In [ ]:
from mcp_integration.tools.base import mcp_registry, MCPToolRegistry

print("Testing MCP Tool Registry\n" + "="*60)

# Get registered tools
tool_names = mcp_registry.get_tool_names()

print(f"✅ Registry contains {len(tool_names)} tools\n")

print("Registered tools:")
for i, name in enumerate(tool_names, 1):
    tool = mcp_registry.get_tool(name)
    print(f"  {i}. {name}")
    print(f"     {tool.description[:80]}...")

print("\n" + "="*60)
print("✅ Registry test complete!")

## Test 2: Individual MCP Tools

Test creating and using individual MCP tools

In [ ]:
from mcp_integration.tools.utilities.datetime_mcp import DateTimeMCPTool
from mcp_integration.tools.google.gmail_mcp import GmailMCPTool
from mcp_integration.tools.google.calendar_mcp import CalendarMCPTool
import json

print("Testing Individual MCP Tools\n" + "="*60)

# Test DateTime Tool
print("\n1. DateTime Tool")
print("-" * 60)
datetime_tool = DateTimeMCPTool()
print(f"Name: {datetime_tool.name}")
print(f"Description: {datetime_tool.description[:100]}...")

# Test execution
result = datetime_tool.execute(operation="get_current")
data = json.loads(result)
print(f"\nTest execution:")
print(f"  Current time: {data.get('current_datetime')}")
print(f"  ✅ Working")

# Test Gmail Tool
print("\n2. Gmail Tool")
print("-" * 60)
gmail_tool = GmailMCPTool()
print(f"Name: {gmail_tool.name}")
print(f"Description: {gmail_tool.description[:100]}...")
print(f"Operations: {', '.join(gmail_tool.get_input_schema()['properties'].keys())}")
print(f"  ✅ Initialized")

# Test Calendar Tool
print("\n3. Calendar Tool")
print("-" * 60)
calendar_tool = CalendarMCPTool()
print(f"Name: {calendar_tool.name}")
print(f"Description: {calendar_tool.description[:100]}...")
print(f"Operations: {', '.join(calendar_tool.get_input_schema()['properties'].keys())}")
print(f"  ✅ Initialized")

print("\n" + "="*60)
print("✅ Individual tool tests complete!")

## Test 3: execute_tool Wrapper

Test the unified execute_tool interface that wraps all MCP tools

In [ ]:
from agents.tool_factory import get_tools

print("Testing execute_tool Wrapper\n" + "="*60)

# Get the execute_tool wrapper
tools = get_tools()
print(f"✅ Loaded {len(tools)} tool(s)")

if len(tools) == 1 and hasattr(tools[0], 'name'):
    execute_tool = tools[0]
    print(f"   Tool name: {execute_tool.name}")
    print(f"   This is the MCP wrapper that delegates to individual tools\n")
    
    # Test calling datetime through execute_tool
    print("Test 1: Call datetime tool through wrapper")
    result = execute_tool.invoke({
        "tool_name": "datetime",
        "parameters": {"operation": "get_current"}
    })
    print(f"  Result: {result[:150]}...")
    print(f"  ✅ Working")
    
    # Test error handling
    print("\nTest 2: Error handling (missing parameters)")
    result = execute_tool.invoke({
        "tool_name": "datetime",
        "parameters": {}  # Missing operation
    })
    print(f"  Result: {result[:150]}...")
    print(f"  ✅ Error handled gracefully")
    
    # Test invalid tool
    print("\nTest 3: Invalid tool name")
    result = execute_tool.invoke({
        "tool_name": "nonexistent_tool",
        "parameters": {}
    })
    print(f"  Result: {result[:150]}...")
    print(f"  ✅ Error handled gracefully")

print("\n" + "="*60)
print("✅ execute_tool wrapper tests complete!")

## Test 4: Tool Input Schemas

Verify that all tools have proper schemas

In [ ]:
from mcp_integration.tools.base import mcp_registry

print("Testing Tool Input Schemas\n" + "="*60)

tool_names = mcp_registry.get_tool_names()

for name in tool_names:
    tool = mcp_registry.get_tool(name)
    schema = tool.get_input_schema()
    
    print(f"\n{name}:")
    print(f"  Type: {schema.get('type')}")
    print(f"  Properties: {list(schema.get('properties', {}).keys())}")
    print(f"  Required: {schema.get('required', [])}")
    print(f"  ✅ Schema valid")

print("\n" + "="*60)
print("✅ All schemas valid!")

## Test 5: LangChain Tool Conversion

Test converting MCP tools to LangChain tools

In [ ]:
from mcp_integration.tools.utilities.datetime_mcp import DateTimeMCPTool

print("Testing LangChain Tool Conversion\n" + "="*60)

# Create MCP tool
mcp_tool = DateTimeMCPTool()
print(f"Original MCP Tool: {mcp_tool.name}")

# Convert to LangChain tool
langchain_tool = mcp_tool.to_langchain_tool()
print(f"\n✅ Converted to LangChain tool")
print(f"   Name: {langchain_tool.name}")
print(f"   Description: {langchain_tool.description[:80]}...")

# Test the LangChain tool
print("\nTesting LangChain tool execution:")
result = langchain_tool.invoke({"operation": "get_current"})
print(f"  Result: {result[:150]}...")
print(f"  ✅ Working")

print("\n" + "="*60)
print("✅ Conversion test complete!")

## Test 6: Tool Modes Comparison

Compare MCP mode vs direct mode (if applicable)

In [ ]:
from agents.tool_factory import get_tool_mode_info
from config import settings

print("Testing Tool Modes\n" + "="*60)

mode_info = get_tool_mode_info()

print(f"Current Mode: {mode_info['mode']}")
print(f"Description: {mode_info['description']}")
print(f"Tool Count: {mode_info['tool_count']}")

print("\n" + "="*60)

if mode_info['mode'] == 'mcp':
    print("✅ MCP Mode Active")
    print("   • All tools wrapped in execute_tool")
    print("   • Unified interface for all operations")
    print("   • Automatic tool registration")
    
elif mode_info['mode'] == 'direct':
    print("✅ Direct Mode Active")
    print("   • Traditional LangChain tools")
    print("   • Direct tool access")
    
elif mode_info['mode'] == 'mcp_client':
    print("✅ MCP Client Mode Active")
    print("   • External MCP servers")
    print("   • Network-based tool execution")

print("\n" + "="*60)
print("✅ Tool mode test complete!")

## Test 7: MCP Protocol Compliance

Verify tools follow MCP protocol standards

In [ ]:
from mcp_integration.tools.base import MCPTool, mcp_registry

print("Testing MCP Protocol Compliance\n" + "="*60)

tool_names = mcp_registry.get_tool_names()

all_compliant = True

for name in tool_names:
    tool = mcp_registry.get_tool(name)
    
    # Check required methods
    has_name = hasattr(tool, 'get_name')
    has_description = hasattr(tool, 'get_description')
    has_schema = hasattr(tool, 'get_input_schema')
    has_execute = hasattr(tool, 'execute')
    
    compliant = all([has_name, has_description, has_schema, has_execute])
    
    status = "✅" if compliant else "❌"
    print(f"{status} {name}")
    
    if not compliant:
        print(f"   Missing methods:")
        if not has_name: print("     - get_name()")
        if not has_description: print("     - get_description()")
        if not has_schema: print("     - get_input_schema()")
        if not has_execute: print("     - execute()")
        all_compliant = False

print("\n" + "="*60)
if all_compliant:
    print("✅ All tools are MCP compliant!")
else:
    print("❌ Some tools are not MCP compliant")
print("="*60)

## Test Summary

In [ ]:
print("\n" + "="*80)
print("📊 MCP INTEGRATION TEST SUMMARY")
print("="*80)
print("")

print("✅ Tested Components:")
print("  1. MCP Tool Registry - Central tool registration")
print("  2. Individual MCP Tools - Tool creation and initialization")
print("  3. execute_tool Wrapper - Unified tool interface")
print("  4. Tool Input Schemas - Schema validation")
print("  5. LangChain Conversion - MCP to LangChain tools")
print("  6. Tool Modes - MCP vs direct modes")
print("  7. Protocol Compliance - MCP standard adherence")
print("")

print("🔑 Key Features:")
print("  • Unified execute_tool wrapper for all tools")
print("  • Automatic tool registration via registry")
print("  • Standardized input/output schemas")
print("  • Seamless LangChain integration")
print("  • Error handling and validation")
print("")

print("🏗️ Architecture:")
print("  MCPTool (base class)")
print("       ↓")
print("  Specific Tools (DateTime, Gmail, Calendar, etc.)")
print("       ↓")
print("  MCPToolRegistry (central registry)")
print("       ↓")
print("  execute_tool (wrapper)")
print("       ↓")
print("  Agent (uses tools)")
print("")

print("="*80)
print("🎉 MCP INTEGRATION FULLY TESTED AND WORKING!")
print("="*80)
print("")
print("All notebooks completed:")
print("  ✅ 01_tools_testing.ipynb - Individual tool tests")
print("  ✅ 02_agents_testing.ipynb - Agent workflows")
print("  ✅ 03_custom_agent_tutorial.ipynb - Build custom agents")
print("  ✅ 04_mcp_integration.ipynb - MCP protocol (this notebook)")
print("="*80)